# May 24:
---

## DIPAM main new features

* There are no more dynamic node/edges fields definitions inside the configuration file. Right now everything is done through the use of <param>. For instance, before adopting this solution we have defined a ‘file_path’ attribute ad-hoc, we can see this as the attribution of a param which let users decide the file path for a given data. The configuration file in this case will be defined this way:
```
“data”:{
    "d-gen-table":{
            "label": "Table",
            "data_class": "table",
            "param": ["p-file"]
        },
...
}
"param": {
        "p-file":{
          "label": "File/s or Directory",
          "value": {}
        },
...
}
```
* DIPAM can detect the inner Loops now.
* A complete different restyling for workflow timeline. In case the workflow completes successfully the results obtained will look like this:
<img src="img/2.png" width="84%">
each timeline block is colored according to the type of node it represents, when a user goes over a block a small textual pop-up will be displayed with the name of the node it represent. In case a node is a "Terminal" node which should return something to the final user: the timeline block will contain a link inside it. E.g. the "save" link of the above figure.  
In case an error occures during the workflow processing, the node which caused the error will be colored in red, and the corresponding label of the error will appear behind it, as it looks like in this figure:
<img src="img/3.png" width="84%">

## DIPAM for CATARSI: Server side

### main.py
This is the main module to execute in order to run DIPAM. It’s based on FLASK, this will run a local web-server instance on the http://localhost:5000/ url. Once this link is used accessed through a web browser the DIPAM web app will be loaded. No other python libraries are needed beside those provided in the flask install package. To install flask just run:  "pip install flask” from your command line. These are the type of URLs handled by the server:

**/**  
Runs DIPAM with it’s default configuration and workflow, these could also correspond to the last configuration and workflow uploaded to the system

**/download/<id\>**   
This call will zip the files of a given node-id, which have been executed on the workflow processing. The zipped file is returned to the user.

**/saveworkflow**  
Let users download and save the corresponding workflow defined in DIPAM locally. 

**/loadworkflow**  
Let users open and load an alternative workflow.


### linker.py
This module will create an inner index for all the “Tool” nodes taking part into the DIPAM workflow.  
For each different “Tool” node, once its execution is done, a new item will be added into the linker dataset.
Each entry of the Linker dataset represent the “Tool” node outputs. For each different “Data” type output produced, the Linker will write down a list of the files associated to such “Data” category.
For instance:
```
{
 t-0001
     path: ‘src/.process-temp/t-0001'
     d-topics: {
         class: ’table'
         files: [<FILE-0>] }
     d-documents: {
         class: ’text'
         files: [<FILE-1>,<FILE-2>] }
  ...
}
```
The upcoming nodes of the workflow which take other “Tool” nodes outputs as their input, will first call the Linker and get their corresponding entry. Then only the compatible “Data” type inputs will be selected. 
For instance: if we the next node to process from the DIPAM workflow is "t-0002”,and one of its incoming edges (inputs) is “t-0001”. Then since “t-0002” works only with “d-documents” data type of inputs, it will only select a subset of files corresponding to such category. (<d-documents>.<files>)


###  data.py
This module process the “Data” nodes. Since these type of nodes could probably point to big datasets, we decided to not re-copy the files inside the DIPAM working directory. We will process (read) the given files directly, and save the results inside a local variable located in this module once the workflow-process is processing a “Data” node. 
This module will be called by the “Tool” nodes in two cases:
1.  If one of their inputs is a “Data” node: in this case the data needed is stored inside this module, and a “Tool” node can demand them.
2. If the inputs are coming from other “Tool” nodes outputs: in this case this module is called in order to read and normalise the input data according to their type.

###  tool.py
This module process the “Tool” nodes. Once DIPAM want to process a “Tool” node from the workflow-process, this module is called as following: Tool.run(). The run method purpose is to recognise the tool type and call the corresponding python module which handles it. For instance: if the node to process is a “t-topic-lda” the corresponding python module to call is textAnalysis.py


## DIAPM first Demo
I have developed a first working Demo example on DIPAM following the CATARSI project objectivies. 
The current Demo can perform a LDA Topic Modelling algorithm on a Textual file and make it available to download.  
To make this we have defined in the configuration file 1 "Data":
```
"d-gen-text":{
            "label": "Text file",
            "data_class": "text",
            "param": ["p-file"]
        }
```
And these two tools:
```
"t-topic-lda": {
            "label": "Topic modeling with LDA",
            "function": "lda",
            "param": ["p-topic"],
            "compatible_input": ["d-gen-text"],
            "output": ["d-gen-table"]
        },
"t-save-files": {
            "label": "Save",
            "function": "save_file",
            "compatible_input": ["d-gen-text","d-gen-table"]
        }
```
Along with the only parameter used:
```
"p-file":{
          "label": "File/s or Directory",
          "value": {}
        },
```

## DHARC presentation
We have officially presented the DHARC and the projects that it manages.

## Attending two Seminaries by Karen Coyle